In [2]:
import math
import time

import torch
from torch import nn
from torch.nn import functional as F

import math
import time

import torch
from torch import nn
from torch.nn import functional as F


/home/kp/miniconda3/envs/gamd5-kp5/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import math
import time

import torch
from torch import nn
from torch.nn import functional as F

import math
import time

import torch
from torch import nn
from torch.nn import functional as F

def diff(ta, tb):
       print(f"delta batch/streaming: {torch.norm(ta - tb) / torch.norm(ta):.2%}")

class Umucs:
    def __init__(self) -> None:            
        self.depth=3
        self.stride_inp=4
        
        self.c1 = nn.Conv1d(in_channels=1, out_channels=48, kernel_size=8, stride=4)
        self.c2 = nn.Conv1d(in_channels=48, out_channels=96, kernel_size=8, stride=4)
        self.c3 = nn.Conv1d(in_channels=96, out_channels=192, kernel_size=8, stride=4)
        self.encoder=nn.ModuleList()
        
        self.encoder.append(self.c1)
        self.encoder.append(self.c2)
        self.encoder.append(self.c3)
        
        # self.inp1= torch.randn(1,296)
        
        self.stride = self.stride_inp ** self.depth
        self.frame_length= self.valid_length(1)

    def get_out(self, length,depth):
        kernel_size=8
        stride=4
        resample=1
        length = math.ceil(length * resample)
        for idx in range(depth):
            length = math.ceil((length - kernel_size) / stride) + 1
            length = max(length, 1)
        return length

    def get_in(self,length,depth):
        """
        Determine the input_length given that we got `length` in the output.
        """
        kernel_size=8
        stride=4
        resample=1
        length = math.ceil(length * resample)
        for idx in range(depth):
            length = (length - 1) * stride + kernel_size
        length = int(math.ceil(length / resample))
        return int(length)

    def valid_length(self,length):
        len = self.get_out(length,self.depth)
        return self.get_in(len,self.depth)
    
    def frm_zmucs(self,inp):
        self.pending = torch.zeros(1, 0)
        self.pending=torch.cat([self.pending,inp],dim=1)
        inp_frames = []
        while self.pending.shape[-1] >= self.frame_length:
            frame = self.pending[:,:self.frame_length]
            inp_frames.append(frame)
            self.pending = self.pending[:,self.stride:] 
        return inp_frames           
        
        
    def feed(self,inp:torch.Tensor,conv_state):
        
        expected_length = self.valid_length(1)
        assert inp.shape[-1] == expected_length
        
        # do_predict(inp)
        
        return_values = [
            inp[:,self.stride:],
        ]
        return return_values

    def framed_inp(self,inp):
        return inp.unfold(1,self.valid_length(1),self.stride).squeeze(0)
    
    def main(self,inp):
        framed_inp = self.framed_inp(inp)
        # zms_fr = torch.stack(self.frm_zmucs(inp)).transpose(0,1)
        # assert torch.allclose(zms_fr,framed_inp) 
        
        self.conv_state = []
        
        out = []
        for each in framed_inp:
            out.append(self.pred_frame(each.unsqueeze(0)))
        
        output = torch.stack(out).transpose(0,3).squeeze(0)
        return output
    
    def pred_frame(self,frame):
        x = frame[None]
        
        first = len(self.conv_state) == 0
        
        stride = self.stride
        
        next_state = []
        for idx, encode in enumerate(self.encoder):
            
            stride = stride // self.stride_inp
            length = x.shape[2]
            
            if not first:
                prev = self.conv_state.pop(0)
                prev = prev[..., stride:]
                
                tgt = (length - 8) // self.stride_inp + 1
                missing = tgt - prev.shape[-1]
                offset = length - 8 - self.stride_inp * (missing - 1)
                x = x[..., offset:]
            
            x = encode(x)
            if not first:
                x = torch.cat([prev, x], -1)

            next_state.append(x)
        
        self.conv_state = next_state
        return x
            
umucs = Umucs()
inp = torch.randn(1,1600)
zmucs_frames = umucs.frm_zmucs(inp)

In [4]:
online_op = umucs.main(inp)
offl_op = umucs.c3(umucs.c2(umucs.c1(inp[None])))
offl_op.shape,online_op.shape

torch.Size([1, 192, 23])

In [5]:
diff(offl_op,online_op)
assert torch.allclose(offl_op,online_op,1e-6,1e-6)

torch.Size([1, 192, 23])

In [38]:
inc = 48
d1 = nn.ConvTranspose1d(in_channels=inc,
                        out_channels=1,
                        kernel_size=8,
                        stride=4,
                        bias=True)
inp2 = torch.arange(0,(inc*400))
out2 = inp2.reshape(1,inc,-1)
out2

tensor([[[    0,     1,     2,  ...,   397,   398,   399],
         [  400,   401,   402,  ...,   797,   798,   799],
         [  800,   801,   802,  ...,  1197,  1198,  1199],
         ...,
         [18000, 18001, 18002,  ..., 18397, 18398, 18399],
         [18400, 18401, 18402,  ..., 18797, 18798, 18799],
         [18800, 18801, 18802,  ..., 19197, 19198, 19199]]])

In [39]:
umucs.c1,inp.shape,out2.shape

(Conv1d(1, 48, kernel_size=(8,), stride=(4,)),
 torch.Size([1, 1600]),
 torch.Size([1, 48, 400]))

In [47]:
inp3 = torch.arange(0,160).reshape(1,1,-1).float()
out3=umucs.c1(inp3)
out3

tensor([[[ -1.2696,  -2.1556,  -3.0416,  ..., -33.1664, -34.0525, -34.9385],
         [ -2.5909,  -3.0116,  -3.4322,  ..., -17.7342, -18.1549, -18.5755],
         [  0.3143,   2.2094,   4.1045,  ...,  68.5379,  70.4330,  72.3281],
         ...,
         [  3.7187,   8.9615,  14.2043,  ..., 192.4600, 197.7028, 202.9456],
         [ -1.2315,  -1.6450,  -2.0584,  ..., -16.1163, -16.5297, -16.9432],
         [  4.4733,   8.1599,  11.8465,  ..., 137.1905, 140.8771, 144.5636]]],
       grad_fn=<ConvolutionBackward0>)

In [48]:
out3.shape

torch.Size([1, 48, 39])